# **Librerias**

In [1]:
# Importamos

import jax
import torch
import warnings
import jax.numpy as jaxnp

from torch.func import jacrev

# Logging 

warnings.filterwarnings('ignore', category = UserWarning)

# Version 

print(f'JAX Version: {jax.__version__}')
print(f'Pytorch Version: {torch.__version__}')

JAX Version: 0.4.34
Pytorch Version: 2.5.1+cpu


# **Hessiana desde Gradientes**

### **Ejercicio #1**

**Funcion:**

$$F(X) = \text{Tr}(X^2B)$$

**Primera Derivada:**

$$F(X) = \text{Tr}(XXB)$$
$$F(X) = \text{Tr}(BXX)$$
$$df(X) = \text{Tr}(d(BXX))$$
$$df(X) = \text{Tr}(BXdX + BdXX + dBXX)$$
$$df(X) = \text{Tr}(BXdX + BdXX)$$
$$df(X) = \text{Tr}(BXdX) +  \text{Tr}(BdXX)$$
$$df(X) = \text{Tr}(BXdX) +  \text{Tr}(XBdX)$$

**Gradiente:** 

$$(\nabla X)^T = BX + XB$$
$$\nabla X = (BX + XB)^T$$

**Segunda Derivada:**

$$df(X) = (BX + XB)^T$$
$$df(X) = X^TB^T + B^TX^T$$
$$df^2(X) = dX^TB^T + X^TdB^T + dB^TX^T + B^TdX^T$$
$$df^2(X) = dX^TB^T + B^TdX^T$$
$$df^2(X) = IdX^TB^T + B^TdX^TI$$
$$df^2(X) = vec(IdX^TB^T) + vec(B^TdX^TI)$$
$$df^2(X) = (B \otimes I) \cdot vec(dX^T) + (I \otimes B^T) \cdot vec(dX^T)$$
$$df^2(X) = (B \otimes I) \cdot K_{m, n} \cdot vec(dX) + (I \otimes B^T) \cdot K_{m, n} \cdot vec(dX)$$
$$df^2(X) = ((B \otimes I) \cdot K_{m, n} + (I \otimes B^T) \cdot K_{m, n}) \cdot vec(dX)$$

**Hessiana:**

$$H = (B \otimes I) \cdot K_{m, n} + (I \otimes B^T) \cdot K_{m, n}$$

In [2]:
# Definimos la funcion matricial

def Fn(X, B):   
    
    output = torch.matmul(X, X) 
    output = torch.matmul(output, B)
    output = torch.trace(output)
    
    return output

# Definimos nuestras matrices de ejemplo

X = torch.tensor([[1.0, 2.0, 7.0], [3.0, 4.0, 9.0], [5.0, 4.0, 1.0]], requires_grad = True)
B = torch.tensor([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0], [5.0, 1.0, 1.0]], requires_grad = False)

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 0), argnums = 0)(X, B)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = hessian.T.permute(2, 3, 0, 1).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[10.,  2.,  5.,  1.,  0.,  0.,  5.,  0.,  0.],
        [ 2.,  0.,  0., 12.,  2.,  5.,  1.,  0.,  0.],
        [ 5.,  0.,  0.,  2.,  0.,  0.,  6.,  2.,  5.],
        [ 1., 12.,  2.,  0.,  1.,  0.,  0.,  5.,  0.],
        [ 0.,  2.,  0.,  1., 14.,  2.,  0.,  1.,  0.],
        [ 0.,  5.,  0.,  0.,  2.,  0.,  1.,  8.,  2.],
        [ 5.,  1.,  6.,  0.,  0.,  1.,  0.,  0.,  5.],
        [ 0.,  0.,  2.,  5.,  1.,  8.,  0.,  0.,  1.],
        [ 0.,  0.,  5.,  0.,  0.,  2.,  5.,  1.,  2.]])


In [3]:
# Definimos la funcion matricial

def Fn(X, B):   
    
    output = jaxnp.matmul(X, X) 
    output = jaxnp.matmul(output, B)
    output = jaxnp.trace(output)
    
    return output

# Definimos nuestras matrices de ejemplo

X = jaxnp.array([[1.0, 2.0, 7.0], [3.0, 4.0, 9.0], [5.0, 4.0, 1.0]])
B = jaxnp.array([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0], [5.0, 1.0, 1.0]])

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 0), argnums = 0)(X, B)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3), (2, 3, 0, 1)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[10.  2.  5.  1.  0.  0.  5.  0.  0.]
 [ 2.  0.  0. 12.  2.  5.  1.  0.  0.]
 [ 5.  0.  0.  2.  0.  0.  6.  2.  5.]
 [ 1. 12.  2.  0.  1.  0.  0.  5.  0.]
 [ 0.  2.  0.  1. 14.  2.  0.  1.  0.]
 [ 0.  5.  0.  0.  2.  0.  1.  8.  2.]
 [ 5.  1.  6.  0.  0.  1.  0.  0.  5.]
 [ 0.  0.  2.  5.  1.  8.  0.  0.  1.]
 [ 0.  0.  5.  0.  0.  2.  5.  1.  2.]]


In [4]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0, 7.0], [3.0, 4.0, 9.0], [5.0, 4.0, 1.0]])
B = torch.tensor([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0], [5.0, 1.0, 1.0]])

I = torch.eye(3)

# Producto Kronecker

term_1 = torch.kron(B, I) 
term_2 = torch.kron(I, B.T.contiguous()) 

# Conmutamos ya que existe vec(dX.T)

n = B.shape[0]
m = X.shape[1] 

commuted_indices = torch.arange(n * m).reshape(n, m).T.reshape(-1)

term1_commuted = term_1[:, commuted_indices]
term2_commuted = term_2[:, commuted_indices]

# Hessiana

hessian = term1_commuted.reshape(term_1.shape) + term2_commuted.reshape(term_2.shape)

# Visualizamos 

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[10.,  2.,  5.,  1.,  0.,  0.,  5.,  0.,  0.],
        [ 2.,  0.,  0., 12.,  2.,  5.,  1.,  0.,  0.],
        [ 5.,  0.,  0.,  2.,  0.,  0.,  6.,  2.,  5.],
        [ 1., 12.,  2.,  0.,  1.,  0.,  0.,  5.,  0.],
        [ 0.,  2.,  0.,  1., 14.,  2.,  0.,  1.,  0.],
        [ 0.,  5.,  0.,  0.,  2.,  0.,  1.,  8.,  2.],
        [ 5.,  1.,  6.,  0.,  0.,  1.,  0.,  0.,  5.],
        [ 0.,  0.,  2.,  5.,  1.,  8.,  0.,  0.,  1.],
        [ 0.,  0.,  5.,  0.,  0.,  2.,  5.,  1.,  2.]])


### **Ejercicio #2**

**Funcion:**

$$F(X) = \text{Tr}(AXBX^TC)$$

**Primera Derivada:**

$$df(X) = \text{Tr}(d(AXBX^TC))$$
$$df(X) = \text{Tr}(AXBX^TdC + AXBdX^TC + AXdBX^TC + AdXBX^TC + dAXBX^TC)$$
$$df(X) = \text{Tr}(AXBdX^TC + AdXBX^TC)$$
$$df(X) = \text{Tr}(AXBdX^TC) +  \text{Tr}(AdXBX^TC)$$
$$df(X) = \text{Tr}(dX^TCAXB) +  \text{Tr}(BX^TCAdX)$$
$$df(X) = \text{Tr}(dX^TCAXB) +  \text{Tr}(BX^TCAdX)$$
$$df(X) = \text{Tr}(B^TX^TA^TC^TdX) +  \text{Tr}(BX^TCAdX)$$

**Gradiente:** 

$$(\nabla X)^T = B^TX^TA^TC^T + BX^TCA$$
$$\nabla X = CAXB + A^TC^TXB^T$$

**Segunda Derivada:**

$$df(X) = CAXB + A^TC^TXB^T$$
$$df^2(X) = dCAXB + CdAXB + CAdXB + CAXdB + dA^TC^TXB^T + A^TdC^TXB^T + A^TC^TdXB^T + A^TC^TXdB^T$$
$$df^2(X) = CAdXB + A^TC^TdXB^T$$
$$df^2(X) =  vec(CAdXB) + vec(A^TC^TdXB^T)$$
$$df^2(X) =  (B^T \otimes CA) \cdot vec(dX) + (B \otimes A^TC^T) \cdot vec(dX)$$
$$df^2(X) =  (B^T \otimes CA + B \otimes A^TC^T) \cdot vec(dX)$$

**Hessiana:**

$$H = B^T \otimes CA + B \otimes A^TC^T$$

In [5]:
# Definimos la funcion matricial

def Fn(X, A, B, C):   
    
    output = torch.matmul(A, X) 
    output = torch.matmul(output, B)
    output = torch.matmul(output, X.T)
    output = torch.matmul(output, C)
    output = torch.trace(output)
    
    return output

# Definimos nuestras matrices de ejemplo

X = torch.tensor([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0]], requires_grad = True)
A = torch.tensor([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0]], requires_grad = False)
B = torch.tensor([[5.0, 2.0, 5.0, 9.0], [1.0, 7.0, 2.0, 9.0], [5.0, 1.0, 1.0, 9.0], [5.0, 1.0, 1.0, 9.0]], requires_grad = False)
C = torch.tensor([[5.0, 4.0], [2.0, 2.0], [8.0, 5.0]], requires_grad = False)

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 0), argnums = 0)(X, A, B, C)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = hessian.T.permute(2, 3, 0, 1).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[290., 250., 390.,  87.,  62., 123., 290., 250., 390., 406., 298., 570.],
        [250., 180., 325.,  88.,  54., 116., 250., 180., 325., 402., 252., 529.],
        [390., 325., 500., 111.,  79., 150., 390., 325., 500., 522., 381., 700.],
        [ 87.,  88., 111., 406., 350., 546.,  87.,  62., 123., 290., 146., 438.],
        [ 62.,  54.,  79., 350., 252., 455.,  88.,  54., 116., 354., 180., 473.],
        [123., 116., 150., 546., 455., 700., 111.,  79., 150., 342., 177., 500.],
        [290., 250., 390.,  87.,  88., 111.,  58.,  50.,  78., 290., 146., 438.],
        [250., 180., 325.,  62.,  54.,  79.,  50.,  36.,  65., 354., 180., 473.],
        [390., 325., 500., 123., 116., 150.,  78.,  65., 100., 342., 177., 500.],
        [406., 402., 522., 290., 354., 342., 290., 354., 342., 522., 450., 702.],
        [298., 252., 381., 146., 180., 177., 146., 180., 177., 450., 324., 585.],
        [570., 529., 700., 438., 473., 500., 438., 473., 500., 702., 58

In [6]:
# Definimos la funcion matricial

def Fn(X, A, B, C):   
    
    output = jaxnp.matmul(A, X) 
    output = jaxnp.matmul(output, B)
    output = jaxnp.matmul(output, X.T)
    output = jaxnp.matmul(output, C)
    output = jaxnp.trace(output)
    
    return output

# Definimos nuestras matrices de ejemplo

X = jaxnp.array([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0]])
A = jaxnp.array([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0]])
B = jaxnp.array([[5.0, 2.0, 5.0, 9.0], [1.0, 7.0, 2.0, 9.0], [5.0, 1.0, 1.0, 9.0], [5.0, 1.0, 1.0, 9.0]])
C = jaxnp.array([[5.0, 4.0], [2.0, 2.0], [8.0, 5.0]])

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 0), argnums = 0)(X, A, B, C)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3), (2, 3, 0, 1)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[290. 250. 390.  87.  62. 123. 290. 250. 390. 406. 298. 570.]
 [250. 180. 325.  88.  54. 116. 250. 180. 325. 402. 252. 529.]
 [390. 325. 500. 111.  79. 150. 390. 325. 500. 522. 381. 700.]
 [ 87.  88. 111. 406. 350. 546.  87.  62. 123. 290. 146. 438.]
 [ 62.  54.  79. 350. 252. 455.  88.  54. 116. 354. 180. 473.]
 [123. 116. 150. 546. 455. 700. 111.  79. 150. 342. 177. 500.]
 [290. 250. 390.  87.  88. 111.  58.  50.  78. 290. 146. 438.]
 [250. 180. 325.  62.  54.  79.  50.  36.  65. 354. 180. 473.]
 [390. 325. 500. 123. 116. 150.  78.  65. 100. 342. 177. 500.]
 [406. 402. 522. 290. 354. 342. 290. 354. 342. 522. 450. 702.]
 [298. 252. 381. 146. 180. 177. 146. 180. 177. 450. 324. 585.]
 [570. 529. 700. 438. 473. 500. 438. 473. 500. 702. 585. 900.]]


In [7]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0]], requires_grad = True)
A = torch.tensor([[5.0, 2.0, 5.0], [1.0, 7.0, 2.0]], requires_grad = False)
B = torch.tensor([[5.0, 2.0, 5.0, 9.0], [1.0, 7.0, 2.0, 9.0], [5.0, 1.0, 1.0, 9.0], [5.0, 1.0, 1.0, 9.0]], requires_grad = False)
C = torch.tensor([[5.0, 4.0], [2.0, 2.0], [8.0, 5.0]], requires_grad = False)

I = torch.eye(3)

# Hessiana

hessian = torch.kron(B.T.contiguous(), torch.matmul(C, A)) + torch.kron(B, torch.matmul(A.T.contiguous(), C.T.contiguous()))

# Visualizamos 

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[290., 250., 390.,  87.,  62., 123., 290., 250., 390., 406., 298., 570.],
        [250., 180., 325.,  88.,  54., 116., 250., 180., 325., 402., 252., 529.],
        [390., 325., 500., 111.,  79., 150., 390., 325., 500., 522., 381., 700.],
        [ 87.,  88., 111., 406., 350., 546.,  87.,  62., 123., 290., 146., 438.],
        [ 62.,  54.,  79., 350., 252., 455.,  88.,  54., 116., 354., 180., 473.],
        [123., 116., 150., 546., 455., 700., 111.,  79., 150., 342., 177., 500.],
        [290., 250., 390.,  87.,  88., 111.,  58.,  50.,  78., 290., 146., 438.],
        [250., 180., 325.,  62.,  54.,  79.,  50.,  36.,  65., 354., 180., 473.],
        [390., 325., 500., 123., 116., 150.,  78.,  65., 100., 342., 177., 500.],
        [406., 402., 522., 290., 354., 342., 290., 354., 342., 522., 450., 702.],
        [298., 252., 381., 146., 180., 177., 146., 180., 177., 450., 324., 585.],
        [570., 529., 700., 438., 473., 500., 438., 473., 500., 702., 58

### **Ejercicio #3**

**Funcion:**

$$F(X) = \, \|X\|_F^2$$

**Primera Derivada:**

$$df(X) = \text{Tr}(d(X X^T))$$
$$df(X) = \text{Tr}(XdX^T + dXX^T)$$
$$df(X) = \text{Tr}(dXX^T + dXX^T)$$
$$df(X) = \text{Tr}(dXX^T) +  \text{Tr}(dXX^T)$$
$$df(X) = \text{Tr}(X^TdX) +  \text{Tr}(X^TdX)$$

**Gradiente:** 

$$(\nabla X)^T = X^T + X^T = 2X^T$$
$$\nabla X = 2X$$

**Segunda Derivada:**

$$df(X) = 2X$$
$$df^2(X) = 2dX$$
$$df^2(X) = 2IdXI$$
$$df^2(X) = 2 \cdot vec(IdXI)$$
$$df^2(X) = 2 \cdot (I \otimes I) \cdot vec(dX)$$

**Hessiana:**

$$H = 2 \cdot (I \otimes I)$$

In [8]:
# Definimos la funcion matricial

def Fn(X):   
    
    output = torch.norm(X)**2 
    
    return output

# Definimos Nuestra Matriz de Ejemplo

X = torch.tensor([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0], [9.0, 5.0, 7.0, 2.0]], requires_grad = True)

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 0), argnums = 0)(X)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = hessian.T.permute(2, 3, 0, 1).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
        [0.

In [9]:
# Definimos la funcion matricial

def Fn(X):   
    
    output = jaxnp.linalg.norm(X)**2 
    
    return output

# Definimos Nuestra Matriz de Ejemplo

X = jaxnp.array([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0], [9.0, 5.0, 7.0, 2.0]])

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 0), argnums = 0)(X)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1], hessian.shape[2] * hessian.shape[3])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3), (2, 3, 0, 1)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]]


In [10]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0, 7.0, 5.0], [3.0, 4.0, 9.0, 1.0], [5.0, 4.0, 1.0, 3.0], [9.0, 5.0, 7.0, 2.0]], requires_grad = True)

I = torch.eye(4)

# Hessiana

hessian = 2 * torch.kron(I, I)

# Visualizamos 

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.],
        [0.

# **Hessiana desde Jacobianas**

### **Ejercicio #1**

**Funcion:**

$$F(X) = X^2$$

**Artificios:** 

$$\frac{dX}{X} = E_{i, j}$$

**Derivada:**

$$d(X) = X dX + dX  X$$
$$d(X) = X  dX  I + I  dX  X$$
$$d(X) = vec(XdXI) + vec(IdXX)$$
$$d(X) = (I \otimes X) \cdot vec(dX) + (X^T \otimes I) \cdot vec(dX)$$
$$d(X) = (I \otimes X + X^T \otimes I) \cdot vec(dX)$$

**Jacobiana:** 

$$J = I \otimes X + X^T \otimes I$$

**Segunda Derivada:** 

$$df(X) = X dX + dX  X$$
$$df(X) = XdX_{1} + dX_{1}X$$
$$df^2(X) = dX_{2}dX_{1} + XdX_{1}^2 + dX_{1}dX_{2} + dX_{1}^2X$$
$$df^2(X) = dX_{2}dX_{1} + dX_{1}dX_{2}$$
$$df^2(X) = E_{i, j}dX_{1} + dX_{1}E_{i, j}$$
$$df^2(X) = E_{i, j}dX_{1}I + IdX_{1}E_{i, j}$$
$$df^2(X) = vec(E_{i, j}dX_{1}I) + vec(IdX_{1}E_{i, j})$$
$$df^2(X) = (I \otimes E_{i, j}) \cdot vec(dX) + (E_{i, j}^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes E_{i, j} + E_{i, j}^T \otimes I) \cdot vec(dX)$$

**Segunda Derivada (Desde Producto Kronecker):** 

$$df(X) = (I \otimes X + X^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (dI \otimes X + I \otimes d(X) + d(X^T) \otimes I + X^T \otimes dI) \cdot vec(dX)$$
$$df^2(X) = (I \otimes d(X) + d(X^T) \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes dX + dX^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes E_{i, j} + E_{i, j}^T \otimes I) \cdot vec(dX)$$

**Hessiana:**

$$H = I \otimes E_{i, j} + E_{i, j}^T \otimes I$$

In [11]:
# Definimos la funcion matricial

def Fn(X):   
    
    output = torch.matmul(X, X)
    
    return output

# Definimos Nuestra Matriz de Ejemplo

X = torch.tensor([[1.0, 2.0], [3.0, 4.0]]) 

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 0), argnums = 0)(X)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = hessian.T.permute(3, 4, 5, 0, 1, 2).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[2., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 2.]])


In [12]:
# Definimos la funcion matricial

def Fn(X):   
    
    output = jaxnp.matmul(X, X)
    
    return output

# Definimos Nuestra Matriz de Ejemplo

X = jaxnp.array([[1.0, 2.0], [3.0, 4.0]]) 

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 0), argnums = 0)(X)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3, 4, 5), (3, 4, 5, 0, 1, 2)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[2. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 2.]]


In [13]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0], [3.0, 4.0]]) 

I = torch.eye(2) 

# Añadimos Dimensiones Adicionales

I = torch.unsqueeze(I, dim = 0)

# Tensor Canonico 

E1 = torch.tensor([[1, 0], [0, 0]], dtype = torch.float32)
E2 = torch.tensor([[0, 0], [1, 0]], dtype = torch.float32)
E3 = torch.tensor([[0, 1], [0, 0]], dtype = torch.float32)
E4 = torch.tensor([[0, 0], [0, 1]], dtype = torch.float32)

E_ij = torch.stack([E1, E2, E3, E4])

# Hessiana

hessian = torch.kron(I, E_ij) + torch.kron(E_ij.permute(0, 2, 1).contiguous(), I)
hessian = hessian.permute(0, 2, 1).reshape(8, 8).T

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[2., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 2.]])


### **Ejercicio #2**

**Funcion:**

$$F(X) = XCX$$

**Artificios:** 

$$\frac{dX}{X} = E_{i, j}$$

**Derivada:**

$$df(X) = XCdX + XdCX + dXCX$$
$$df(X) = XCdX + dXCX$$
$$df(X) = XCdXI + IdXCX$$
$$df(X) = vec(XCdXI) + vec(IdXCX)$$
$$df(X) = (I \otimes XC) \cdot vec(dX) + ((CX)^T \otimes I) \cdot vec(dX)$$
$$df(X) = (I \otimes XC + (CX)^T \otimes I) \cdot vec(dX)$$

**Jacobiana:** 

$$J = I \otimes XC + (CX)^T \otimes I$$

**Segunda Derivada:** 

$$df(X) = XCdX + dXCX$$
$$df(X) = XCdX_{1} + dX_{1}CX$$
$$df^2(X) = dX_{2}CdX_{1} + XdCdX_{1} + XCdX_{1}^2 + dX_{1}^2CX + dX_{1}dCX + dX_{1}CdX_{2}$$
$$df^2(X) = dX_{2}CdX_{1} + dX_{1}CdX_{2}$$
$$df^2(X) = E_{i, j}CdX_{1} + dX_{1}CE_{i, j}$$
$$df^2(X) = E_{i, j}CdX_{1}I + IdX_{1}CE_{i, j}$$
$$df^2(X) = vec(E_{i, j}CdX_{1}I) + vec(IdX_{1}CE_{i, j})$$
$$df^2(X) = (I \otimes E_{i, j}C) \cdot vec(dX) + (E_{i, j}^TC^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes E_{i, j}C + E_{i, j}^TC^T \otimes I) \cdot vec(dX)$$

**Segunda Derivada (Desde Producto Kronecker):** 

$$df(X) = (I \otimes XC + (CX)^T \otimes I) \cdot vec(dX)$$
$$df(X) = (I \otimes XC + X^TC^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (dI \otimes XC + I \otimes d(XC) + d(X^TC^T) \otimes I + X^TC^T \otimes dI) \cdot vec(dX)$$
$$df^2(X) = (I \otimes d(XC) + d(X^TC^T) \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes dXC + dX^TC^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (I \otimes E_{i, j}C + E_{i, j}^TC^T \otimes I) \cdot vec(dX)$$

**Hessiana:**

$$H = I \otimes E_{i, j}C + E_{i, j}^TC^T \otimes I$$

In [14]:
# Definimos la funcion matricial

def Fn(X, C):   
    
    output = torch.matmul(X, C)
    output = torch.matmul(output, X)
    
    return output

# Definimos nuestras matrices de ejemplo

X = torch.tensor([[1.0, 2.0], [3.0, 4.0]]) 
C = torch.tensor([[8.0, 9.0], [5.0, 7.0]]) 

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 0), argnums = 0)(X, C)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = hessian.T.permute(3, 4, 5, 0, 1, 2).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[16.,  5.,  9.,  7.,  5.,  0.,  0.,  0.],
        [ 0.,  0.,  8.,  0.,  0.,  0.,  5.,  0.],
        [ 0.,  8.,  0.,  0.,  8., 10.,  9.,  7.],
        [ 0.,  0.,  0.,  8.,  0.,  0.,  0.,  5.],
        [ 9.,  0.,  0.,  0.,  7.,  0.,  0.,  0.],
        [ 8.,  5., 18.,  7.,  0.,  0.,  7.,  0.],
        [ 0.,  9.,  0.,  0.,  0.,  7.,  0.,  0.],
        [ 0.,  0.,  0.,  9.,  8.,  5.,  9., 14.]])


In [15]:
# Definimos la funcion matricial

def Fn(X, C):   
    
    output = jaxnp.matmul(X, C)
    output = jaxnp.matmul(output, X)
    
    return output

# Definimos nuestras matrices de ejemplo

X = jaxnp.array([[1.0, 2.0], [3.0, 4.0]]) 
C = jaxnp.array([[8.0, 9.0], [5.0, 7.0]]) 

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el primer argumento 0 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 0), argnums = 0)(X, C)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3, 4, 5), (3, 4, 5, 0, 1, 2)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[16.  5.  9.  7.  5.  0.  0.  0.]
 [ 0.  0.  8.  0.  0.  0.  5.  0.]
 [ 0.  8.  0.  0.  8. 10.  9.  7.]
 [ 0.  0.  0.  8.  0.  0.  0.  5.]
 [ 9.  0.  0.  0.  7.  0.  0.  0.]
 [ 8.  5. 18.  7.  0.  0.  7.  0.]
 [ 0.  9.  0.  0.  0.  7.  0.  0.]
 [ 0.  0.  0.  9.  8.  5.  9. 14.]]


In [16]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0], [3.0, 4.0]]) 
C = torch.tensor([[8.0, 9.0], [5.0, 7.0]]) 

I = torch.eye(2) 

# Añadimos Dimensiones Adicionales

C = torch.unsqueeze(C, dim = 0)
I = torch.unsqueeze(I, dim = 0)

# Tensor Canonico

E1 = torch.tensor([[1, 0], [0, 0]], dtype = torch.float32)
E2 = torch.tensor([[0, 0], [1, 0]], dtype = torch.float32)
E3 = torch.tensor([[0, 1], [0, 0]], dtype = torch.float32)
E4 = torch.tensor([[0, 0], [0, 1]], dtype = torch.float32)

E_ij = torch.stack([E1, E2, E3, E4])

# Hessiana

hessian = torch.kron(I, E_ij @ C) + torch.kron(E_ij.permute(0, 2, 1) @ C.permute(0, 2, 1), I)
hessian = hessian.permute(0, 2, 1).reshape(8, 8).T

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[16.,  5.,  9.,  7.,  5.,  0.,  0.,  0.],
        [ 0.,  0.,  8.,  0.,  0.,  0.,  5.,  0.],
        [ 0.,  8.,  0.,  0.,  8., 10.,  9.,  7.],
        [ 0.,  0.,  0.,  8.,  0.,  0.,  0.,  5.],
        [ 9.,  0.,  0.,  0.,  7.,  0.,  0.,  0.],
        [ 8.,  5., 18.,  7.,  0.,  0.,  7.,  0.],
        [ 0.,  9.,  0.,  0.,  0.,  7.,  0.,  0.],
        [ 0.,  0.,  0.,  9.,  8.,  5.,  9., 14.]])


### **Ejercicio #3**

**Funcion:**

$$F(X) = XAX^TB$$

**Artificios:** 

$$\frac{dX}{X} = E_{i, j}$$

**Derivada:**

$$df(X) = XAX^TdB + XAdX^TB + XdAX^TB + dXAX^TB$$
$$df(X) = XAdX^TB + dXAX^TB$$
$$df(X) = vec(XAdX^TB) + vec(dXAX^TB)$$
$$df(X) = (B^T \otimes XA) \cdot vec(dX^T) + ((AX^TB)^T \otimes I) \cdot vec(dX)$$
$$df(X) = (B^T \otimes XA) \cdot K_{m, n} \cdot vec(dX) + ((AX^TB)^T \otimes I) \cdot vec(dX)$$
$$df(X) = ((B^T \otimes XA) \cdot K_{m, n} +  ((AX^TB)^T \otimes I))\cdot vec(dX)$$

**Jacobiana:** 

$$J = (B^T \otimes XA) \cdot K_{m, n} +  (AX^TB)^T \otimes I$$

**Segunda Derivada:** 

$$df(X) = XAdX^TB + dXAX^TB$$
$$df(X) = XAdX_{1}^TB + dX_{1}AX^TB$$
$$df^2(X) = dX_{2}AdX_{1}^TB + XdAdX_{1}^TB + XA(dX_{1}^2)^TB + XAdX_{1}^TdB + dX_{1}^2AX^TB + dX_{1}dAX^TB + dX_{1}AdX_{2}^TB + dX_{1}AX^TdB$$
$$df^2(X) = dX_{2}AdX_{1}^TB + dX_{1}AdX_{2}^TB$$
$$df^2(X) = E_{i, j}AdX_{1}^TB + dX_{1}AE_{i, j}^TB$$
$$df^2(X) = vec(E_{i, j}AdX_{1}^TB) + vec(dX_{1}AE_{i, j}^TB)$$
$$df^2(X) = (B^T \otimes E_{i, j}A) \cdot vec(dX^T) + (B^TE_{i, j}A^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = (B^T \otimes E_{i, j}A) \cdot K_{m, n} \cdot vec(dX) + (B^TE_{i, j}A^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = ((B^T \otimes E_{i, j}A) \cdot K_{m, n} + B^TE_{i, j}A^T  \otimes I) \cdot vec(dX)$$

**Segunda Derivada (Desde Producto Kronecker):** 

$$df(X) = ((B^T \otimes XA) \cdot K_{m, n} +  ((AX^TB)^T \otimes I))\cdot vec(dX)$$
$$df^2(X) = ((dB^T \otimes XA + B^T \otimes d(XA)) \cdot K_{m, n} + d(AX^TB)^T \otimes I + (AX^TB)^T \otimes dI) \cdot vec(dX)$$
$$df^2(X) = ((B^T \otimes d(XA)) \cdot K_{m, n} + d(AX^TB)^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = ((B^T \otimes dXA) \cdot K_{m, n} + (AdX^TB)^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = ((B^T \otimes E_{i, j}A) \cdot K_{m, n} + (AE_{i, j}^TB)^T \otimes I) \cdot vec(dX)$$
$$df^2(X) = ((B^T \otimes E_{i, j}A) \cdot K_{m, n} + B^TE_{i, j}A^T \otimes I) \cdot vec(dX)$$

**Hessiana:**

$$H = (B^T \otimes E_{i, j}A) \cdot K_{m, n} + B^TE_{i, j}A^T  \otimes I$$

In [17]:
# Definimos la funcion matricial

def Fn(A, X, B):   
    
    output = torch.matmul(X, A)
    output = torch.matmul(output, X.T)
    output = torch.matmul(output, B)
    
    return output

# Definimos nuestras matrices de ejemplo

X = torch.tensor([[1.0, 2.0], [3.0, 4.0], [9.0, 5.0]]) 
A = torch.tensor([[8.0, 9.0], [5.0, 7.0]]) 
B = torch.tensor([[8.0, 9.0, 6.0], [5.0, 7.0, 1.0], [7.0, 6.0, 4.0]])

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el segundo argumento 1 --> X) (backward-mode)

hessian = jacrev(jacrev(Fn, argnums = 1), argnums = 1)(A, X, B)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = hessian.T.permute(3, 4, 5, 0, 1, 2).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
tensor([[128., 112.,  40.,  25.,  56.,  35., 112., 112.,  45.,  35.,  63.,  49.],
        [  0.,   0.,  64.,  72.,   0.,   0.,   0.,   0.,  40.,  56.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  64.,  72.,   0.,   0.,   0.,   0.,  40.,  56.],
        [144., 126.,  56.,  35.,  48.,  30., 126., 126.,  63.,  49.,  54.,  42.],
        [  0.,   0.,  72.,  81.,   0.,   0.,   0.,   0.,  45.,  63.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  72.,  81.,   0.,   0.,   0.,   0.,  45.,  63.],
        [ 96.,  84.,   8.,   5.,  32.,  20.,  84.,  84.,   9.,   7.,  36.,  28.],
        [  0.,   0.,  48.,  54.,   0.,   0.,   0.,   0.,  30.,  42.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  48.,  54.,   0.,   0.,   0.,   0.,  30.,  42.],
        [ 40.,  45.,   0.,   0.,   0.,   0.,  25.,  35.,   0.,   0.,   0.,   0.],
        [ 64.,  40.,  80.,  70.,  56.,  35.,  72.,  56.,  70.,  70.,  63.,  49.],
        [  0.,   0.,   0.,   0.,  40.,  45.,   0.,   0.,   0.,   0.,  2

In [18]:
# Definimos la funcion matricial

def Fn(A, X, B):   
    
    output = jaxnp.matmul(X, A)
    output = jaxnp.matmul(output, X.T)
    output = jaxnp.matmul(output, B)
    
    return output

# Definimos nuestras matrices de ejemplo

X = jaxnp.array([[1.0, 2.0], [3.0, 4.0], [9.0, 5.0]]) 
A = jaxnp.array([[8.0, 9.0], [5.0, 7.0]]) 
B = jaxnp.array([[8.0, 9.0, 6.0], [5.0, 7.0, 1.0], [7.0, 6.0, 4.0]])

# Calculamos la Matriz Hessiana (Indicamos con respecto a que argumento derivamos dos veces en nuestro caso el segundp argumento 1 --> X) (backward-mode)

hessian = jax.jacrev(jax.jacrev(Fn, argnums = 1), argnums = 1)(A, X, B)

# Redimensionamos

new_shape = (hessian.shape[0] * hessian.shape[1] * hessian.shape[2], hessian.shape[3] * hessian.shape[4] * hessian.shape[5])

hessian = jaxnp.moveaxis(hessian.T, (0, 1, 2, 3, 4, 5), (3, 4, 5, 0, 1, 2)).reshape(new_shape)

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')

La Hessiana Respecto a X: 
[[128. 112.  40.  25.  56.  35. 112. 112.  45.  35.  63.  49.]
 [  0.   0.  64.  72.   0.   0.   0.   0.  40.  56.   0.   0.]
 [  0.   0.   0.   0.  64.  72.   0.   0.   0.   0.  40.  56.]
 [144. 126.  56.  35.  48.  30. 126. 126.  63.  49.  54.  42.]
 [  0.   0.  72.  81.   0.   0.   0.   0.  45.  63.   0.   0.]
 [  0.   0.   0.   0.  72.  81.   0.   0.   0.   0.  45.  63.]
 [ 96.  84.   8.   5.  32.  20.  84.  84.   9.   7.  36.  28.]
 [  0.   0.  48.  54.   0.   0.   0.   0.  30.  42.   0.   0.]
 [  0.   0.   0.   0.  48.  54.   0.   0.   0.   0.  30.  42.]
 [ 40.  45.   0.   0.   0.   0.  25.  35.   0.   0.   0.   0.]
 [ 64.  40.  80.  70.  56.  35.  72.  56.  70.  70.  63.  49.]
 [  0.   0.   0.   0.  40.  45.   0.   0.   0.   0.  25.  35.]
 [ 56.  63.   0.   0.   0.   0.  35.  49.   0.   0.   0.   0.]
 [ 72.  45. 112.  98.  48.  30.  81.  63.  98.  98.  54.  42.]
 [  0.   0.   0.   0.  56.  63.   0.   0.   0.   0.  35.  49.]
 [  8.   9.   0.   0.   0.  

In [19]:
# Definimos Nuestras Matrices

X = torch.tensor([[1.0, 2.0], [3.0, 4.0], [9.0, 5.0]]) 
A = torch.tensor([[8.0, 9.0], [5.0, 7.0]]) 
B = torch.tensor([[8.0, 9.0, 6.0], [5.0, 7.0, 1.0], [7.0, 6.0, 4.0]])

I = torch.eye(3) 

# Añadimos Dimensiones Adicionales

A = torch.unsqueeze(A, dim = 0)
B = torch.unsqueeze(B, dim = 0)
I = torch.unsqueeze(I, dim = 0)

# Tensor Canonico

E1 = torch.tensor([[1, 0], [0, 0], [0, 0]], dtype = torch.float32)
E2 = torch.tensor([[0, 0], [1, 0], [0, 0]], dtype = torch.float32)
E3 = torch.tensor([[0, 0], [0, 0], [1, 0]], dtype = torch.float32)
E4 = torch.tensor([[0, 1], [0, 0], [0, 0]], dtype = torch.float32)
E5 = torch.tensor([[0, 0], [0, 1], [0, 0]], dtype = torch.float32)
E6 = torch.tensor([[0, 0], [0, 0], [0, 1]], dtype = torch.float32)

E_ij = torch.stack([E1, E2, E3, E4, E5, E6])

# Producto Kronecker

term_1 = torch.kron(B.permute(0, 2, 1).contiguous(), E_ij @ A)
term_2 = torch.kron(B.permute(0, 2, 1).contiguous() @ E_ij @ A.permute(0, 2, 1).contiguous(), I)

# Conmutamos ya que existe vec(dX.T)

n = A.shape[2]
m = X.shape[0] 

commuted_indices = torch.arange(n * m).reshape(m, n).T.reshape(-1)

term1_commuted = term_1[:, :, commuted_indices]

# Hessiana

hessian = term1_commuted.reshape(term_1.shape) + term_2
hessian = hessian.permute(0, 2, 1).reshape(12, 27).T

# Visualizamos

print(f'La Hessiana Respecto a X: \n{hessian}')


La Hessiana Respecto a X: 
tensor([[128., 112.,  40.,  25.,  56.,  35., 112., 112.,  45.,  35.,  63.,  49.],
        [  0.,   0.,  64.,  72.,   0.,   0.,   0.,   0.,  40.,  56.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  64.,  72.,   0.,   0.,   0.,   0.,  40.,  56.],
        [144., 126.,  56.,  35.,  48.,  30., 126., 126.,  63.,  49.,  54.,  42.],
        [  0.,   0.,  72.,  81.,   0.,   0.,   0.,   0.,  45.,  63.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  72.,  81.,   0.,   0.,   0.,   0.,  45.,  63.],
        [ 96.,  84.,   8.,   5.,  32.,  20.,  84.,  84.,   9.,   7.,  36.,  28.],
        [  0.,   0.,  48.,  54.,   0.,   0.,   0.,   0.,  30.,  42.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  48.,  54.,   0.,   0.,   0.,   0.,  30.,  42.],
        [ 40.,  45.,   0.,   0.,   0.,   0.,  25.,  35.,   0.,   0.,   0.,   0.],
        [ 64.,  40.,  80.,  70.,  56.,  35.,  72.,  56.,  70.,  70.,  63.,  49.],
        [  0.,   0.,   0.,   0.,  40.,  45.,   0.,   0.,   0.,   0.,  2